In [1]:
!pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score

/tmp/ipykernel_240/4092108926.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"FACT_CUSTOMER_ORDERS"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [ ]:
data = df[['PRODUCT_ID','ORDER_DATE','ORDER_QTY']]

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data['ORDER_DATE'] = pd.to_datetime(data['ORDER_DATE'])

In [ ]:
data = data.set_index('ORDER_DATE')

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.sort_index(inplace=True)

In [ ]:
# Group data by 'product_id'
grouped_data = data.groupby('PRODUCT_ID')

In [ ]:
# Initialize an empty DataFrame for forecast results
test_results = pd.DataFrame(columns=['product_id','order_date', 'test_forecast_orders','rse', 'rmse','r2_score'])

In [ ]:
# Initialize an empty DataFrame for forecast results
forecast_results = pd.DataFrame(columns=['product_id','order_date', 'forecast_order_qty'])

In [ ]:
for product_id, group in grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)

    
    # Split data into train and test sets (80% train, 20% test)
    train_size = int(0.8 * len(group))
    train_data, test_data = group.iloc[:train_size], group.iloc[train_size:]
    
    # Create SARIMAX model
    model = SARIMAX(train_data['ORDER_QTY'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    
    # Forecast the next period
    test_model = model_fit.get_forecast(steps=len(test_data))
    test_df = test_model.predicted_mean

    # Evaluate the model
    rse = np.sqrt(((test_df - test_data) ** 2).sum().sum() / (test_data.shape[0] * test_data.shape[1] - len(model_fit.params)))
    rmse = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], test_df))
    r2 = r2_score(test_data['ORDER_QTY'], test_df)

    # Convert index to date format
    test_df.index = pd.to_datetime(test_df.index)
    date_list = test_df.index.strftime('%Y-%m-%d').to_list()
    
    new_row = {
        'product_id': product_id,
        'order_date': date_list,
        'test_forecast_orders': test_df,
        'rse': rse,
        'rmse': rmse,
        'r2_score': r2
    }
    
    df_new = pd.DataFrame(new_row)
    test_results = pd.concat([test_results, df_new], ignore_index=True)
    
    # Forecast future values
    forecast = model_fit.get_forecast(steps=25)
    forecast_df = forecast.predicted_mean
    
    # Convert index to date format
    forecast_df.index = pd.to_datetime(forecast_df.index)
    date_list2 = forecast_df.index.strftime('%Y-%m-%d').to_list()
    
    new_row2 = {
        'product_id': product_id,
        'order_date': date_list2,
        'forecast_order_qty': forecast_df,
    }
    
    df_new2 = pd.DataFrame(new_row2)
    forecast_results = pd.concat([forecast_results, df_new2], ignore_index=True)


In [ ]:
test_results

In [ ]:
test_df.index

In [ ]:
data

In [ ]:
data.index

In [ ]:
grouped_data.index

In [ ]:
for product_id, group in grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)

    print({group.index})

In [ ]:
for product_id, group in grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)

    train_size = int(0.8 * len(group))
    train_data, test_data = group.iloc[:train_size], group.iloc[train_size:]

    #print(f'Train_data:{train_data.index}')
    #print(f'Test_data:{test_data.index}')

    model = SARIMAX(train_data['ORDER_QTY'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    
    # Forecast the next period
    test_model = model_fit.get_forecast(steps=len(test_data))
    test_df = test_model.predicted_mean

    #date_list = test_df.index.to_list()
   
    print(f'Datelist:{test_df.index}')


In [ ]:
test_df

In [ ]:
test_df.info

In [ ]:
test_df.info()

In [ ]:
test_model.info()

In [ ]:
test_model

In [ ]:
test_df.index

In [ ]:
date_list=pd.to_datetime(test_df.index)

In [ ]:
date_list

In [ ]:
test_model.index()

In [ ]:
for product_id, group in grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)

    
    # Split data into train and test sets (80% train, 20% test)
    train_size = int(0.8 * len(group))
    train_data, test_data = group.iloc[:train_size], group.iloc[train_size:]
    
    # Create SARIMAX model
    model = SARIMAX(train_data['ORDER_QTY'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    
    # Forecast the next period
    test_model = model_fit.get_forecast(steps=len(test_data))
    test_df = test_model.predicted_mean

    # Evaluate the model
    rse = np.sqrt(((test_df - test_data) ** 2).sum().sum() / (test_data.shape[0] * test_data.shape[1] - len(model_fit.params)))
    rmse = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], test_df))
    r2 = r2_score(test_data['ORDER_QTY'], test_df)

    date_list = test_data.index.to_list()
    
    new_row = {
        'product_id': product_id,
        'order_date': date_list,
        'test_forecast_orders': test_df,
        'rse': rse,
        'rmse': rmse,
        'r2_score': r2
    }
    
    df_new = pd.DataFrame(new_row)
    test_results = pd.concat([test_results, df_new], ignore_index=True)
